In [1]:
from onautilus.onautilus import ONAUTILUS
from onautilus.preferential_func_eval import preferential_func_eval as pfe

import numpy as np
import pandas as pd

from desdeo_problem.Problem import DataProblem
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

from desdeo_emo.EAs.RVEA import RVEA

from pygmo import fast_non_dominated_sorting as nds

from UI.pages.tools import create_navigator_plot, extend_navigator_plot
import plotly.express as ex

from onautilus.problems import vehicle_crash_worthiness

In [2]:
import plotly.io as pio

pio.kaleido.scope.default_width = 1000
pio.kaleido.scope.default_height = 400

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv("../data/veh_lhs.csv")
data.rename(columns={
    "weight": "f1",
    "acceleration characteristics": "f2",
    "toe-board instruction": "f3"},
            inplace=True
           )
var_names = data.columns[0:5].tolist()
obj_names = data.columns[5:].tolist()

bounds = data.describe()[var_names].loc[["min", "max"]].rename(index={"min": "lower_bound", "max": "upper_bound"})

In [5]:
data

,x1,x2,x3,x4,x5,f1,f2,f3
0,1.630568,1.164042,1.044959,2.597417,2.597592,1683.233620,10.854482,0.081062
1,1.174863,1.351114,2.935511,1.768088,2.341108,1683.685034,8.638321,0.112737
2,2.958312,2.926587,1.768924,2.099489,1.687858,1685.865477,9.512983,0.103124
3,1.046416,2.294535,2.940803,2.807601,2.762791,1695.502498,10.406023,0.116614
4,1.907029,1.670924,1.324802,1.098803,1.390753,1669.391926,8.573261,0.115932
...,...,...,...,...,...,...,...,...
95,1.415994,2.328067,2.404775,2.821029,1.724818,1689.481062,10.799269,0.127517
96,1.699596,1.331335,2.099911,2.655063,1.164170,1682.662595,10.611373,0.110092
97,1.258890,2.478506,1.689111,1.451819,2.653414,1679.755866,8.441823,0.116881
98,2.600078,2.086669,2.566626,1.802794,1.138428,1681.976194,8.856274,0.170954


In [6]:
problem = DataProblem(
    data=data,
    objective_names=obj_names,
    variable_names=var_names,
    bounds=bounds
)

In [7]:
problem.train(GaussianProcessRegressor, {"kernel":Matern(nu=3/2)})

In [8]:
optimizer = RVEA(
    problem,
    use_surrogates=True,
    **{"n_iterations": 10, "n_gen_per_iter": 20, "selection_type": "optimistic"}
)

In [9]:
while optimizer.continue_evolution():
    optimizer.iterate()

In [10]:
individuals = optimizer.population.individuals

objectives = (
        optimizer.population.objectives - optimizer.population.uncertainity
    )
    
non_dom_indices = nds(objectives)[0][0]

individuals = individuals[non_dom_indices]
objectives = objectives[non_dom_indices]
optimistic_data = pd.DataFrame(
    np.hstack((individuals, objectives)),
    columns=problem.variable_names + problem.objective_names,
)

In [11]:
method = ONAUTILUS(
    known_data=data[obj_names].values,
    optimistic_data=optimistic_data[obj_names].values,
    objective_names=obj_names,
    max_multiplier=[1,1,1],
    num_steps=100
)

ranges_plot_request, _, preference_request = method.requests()
ranges_plot_request.content["total_steps"] = 80
nav_figures = [
            create_navigator_plot(
                ranges_plot_request,
                objective_name,
                have_x_label=True, 
                legend=True, 
                have_y_label=True
            )
            for objective_name in obj_names
        ]

In [12]:
preference_request.response = pd.DataFrame(
        [[1669.39, 7.16433, 0.059]],
        columns=preference_request.content["dimensions_data"].columns,
    )

In [13]:
step_max = 39

while method.steps_taken < step_max:
    ranges_plot_request, _, preference_request = method.iterate(preference_request)
    ranges_plot_request.content["total_steps"] = 80
    nav_figures = [
                extend_navigator_plot(ranges_plot_request, objective_name, nav_figure)
                for nav_figure, objective_name in zip(nav_figures, obj_names)
            ]

In [21]:
nav_figures = [
    nav_figure.update_layout(font_size=15)
    for nav_figure, objective_name in zip(nav_figures, obj_names)
]

In [22]:
[
    nav_figure.write_image(f"images/{objective_name}_1.pdf")
    for nav_figure, objective_name in zip(nav_figures, obj_names)
]

[None, None, None]

In [23]:
ideal_current = (
        ranges_plot_request.content["dimensions_data"].loc["ideal"].values
        * problem._max_multiplier
    )
nadir_current = (
        ranges_plot_request.content["dimensions_data"].loc["nadir"].values
        * problem._max_multiplier
    )

In [24]:
x_new = np.atleast_2d(
        pfe(
            problem,
            optimistic_data,
            ideal_current,
            nadir_current,
            reference_point=np.asarray((1669.39, 7.09049, 0.07099)),
        ).result.xbest
    )

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=824242, Thu Nov 19 20:08:33 2020)


In [25]:
y_new = vehicle_crash_worthiness(x_new)

In [26]:
y_new.tolist()

[[1672.447135406452, 7.23394391073328, 0.08145964973404053]]

In [27]:
x = data[var_names].values
y = data[obj_names].values
x = np.vstack((x, x_new))
y = np.vstack((y, y_new))
data = np.hstack((x, y))
data = pd.DataFrame(data, columns=var_names + obj_names)

In [28]:
data

,x1,x2,x3,x4,x5,f1,f2,f3
0,1.630568,1.164042,1.044959,2.597417,2.597592,1683.233620,10.854482,0.081062
1,1.174863,1.351114,2.935511,1.768088,2.341108,1683.685034,8.638321,0.112737
2,2.958312,2.926587,1.768924,2.099489,1.687858,1685.865477,9.512983,0.103124
3,1.046416,2.294535,2.940803,2.807601,2.762791,1695.502498,10.406023,0.116614
4,1.907029,1.670924,1.324802,1.098803,1.390753,1669.391926,8.573261,0.115932
...,...,...,...,...,...,...,...,...
96,1.699596,1.331335,2.099911,2.655063,1.164170,1682.662595,10.611373,0.110092
97,1.258890,2.478506,1.689111,1.451819,2.653414,1679.755866,8.441823,0.116881
98,2.600078,2.086669,2.566626,1.802794,1.138428,1681.976194,8.856274,0.170954
99,1.860919,1.898737,2.611184,2.668343,1.866846,1689.929966,10.352398,0.125945


In [29]:
problem = DataProblem(
    data=data,
    objective_names=obj_names,
    variable_names=var_names,
    bounds=bounds
)
problem.train(GaussianProcessRegressor, {"kernel":Matern(nu=3/2)})

optimizer = RVEA(
    problem,
    use_surrogates=True,
    **{"n_iterations": 10, "n_gen_per_iter": 20, "selection_type": "optimistic"}
)

while optimizer.continue_evolution():
    optimizer.iterate()
    
individuals = optimizer.population.individuals

objectives = (
        optimizer.population.objectives - optimizer.population.uncertainity
    )
    
non_dom_indices = nds(objectives)[0][0]

individuals = individuals[non_dom_indices]
objectives = objectives[non_dom_indices]
optimistic_data = pd.DataFrame(
    np.hstack((individuals, objectives)),
    columns=problem.variable_names + problem.objective_names,
)

method = ONAUTILUS(
    known_data=data[obj_names].values,
    optimistic_data=optimistic_data[obj_names].values,
    objective_names=obj_names,
    max_multiplier=[1,1,1],
    num_steps=100
)

ranges_plot_request, _, preference_request = method.requests()

ranges_plot_request.content["total_steps"] = 80

nav_figures = [
            create_navigator_plot(
                ranges_plot_request,
                objective_name,
                have_x_label=True, 
                legend=True, 
                have_y_label=True
            )
            for objective_name in obj_names
        ]

In [30]:
preference_request.response = pd.DataFrame(
        [(1669.39, 7.09049, 0.07099)],
        columns=preference_request.content["dimensions_data"].columns,
    )

In [31]:
step_max = 25

while method.steps_taken < step_max:
    ranges_plot_request, _, preference_request = method.iterate(preference_request)
    ranges_plot_request.content["total_steps"] = 80
    nav_figures = [
                extend_navigator_plot(ranges_plot_request, objective_name, nav_figure)
                for nav_figure, objective_name in zip(nav_figures, obj_names)
            ]

In [32]:
ideal_current = (
        ranges_plot_request.content["dimensions_data"].loc["ideal"].values
        * problem._max_multiplier
    )
nadir_current = (
        ranges_plot_request.content["dimensions_data"].loc["nadir"].values
        * problem._max_multiplier
    )

In [33]:
x_new = np.atleast_2d(
        pfe(
            problem,
            optimistic_data,
            ideal_current,
            nadir_current,
            reference_point=np.asarray((1661.58, 7.09049, 0.07099)),
        ).result.xbest
    )

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=923460, Thu Nov 19 20:09:05 2020)


In [34]:
y_new = vehicle_crash_worthiness(x_new)
y_new.tolist()

[[1666.8648228135469, 7.064629594531506, 0.09251168839342443]]

In [35]:
x = data[var_names].values
y = data[obj_names].values
x = np.vstack((x, x_new))
y = np.vstack((y, y_new))
data = np.hstack((x, y))
data = pd.DataFrame(data, columns=var_names + obj_names)

In [36]:
problem = DataProblem(
    data=data,
    objective_names=obj_names,
    variable_names=var_names,
    bounds=bounds
)
problem.train(GaussianProcessRegressor, {"kernel":Matern(nu=3/2)})

optimizer = RVEA(
    problem,
    use_surrogates=True,
    **{"n_iterations": 10, "n_gen_per_iter": 20, "selection_type": "optimistic"}
)

while optimizer.continue_evolution():
    optimizer.iterate()
    
individuals = optimizer.population.individuals

objectives = (
        optimizer.population.objectives - optimizer.population.uncertainity
    )
    
non_dom_indices = nds(objectives)[0][0]

individuals = individuals[non_dom_indices]
objectives = objectives[non_dom_indices]
optimistic_data = pd.DataFrame(
    np.hstack((individuals, objectives)),
    columns=problem.variable_names + problem.objective_names,
)

method = ONAUTILUS(
    known_data=data[obj_names].values,
    optimistic_data=optimistic_data[obj_names].values,
    objective_names=obj_names,
    max_multiplier=[1,1,1],
    num_steps=100
)

ranges_plot_request, _, preference_request = method.requests()
ranges_plot_request.content["total_steps"] = 80

nav_figures = [
            create_navigator_plot(
                ranges_plot_request,
                objective_name,
                have_x_label=True, 
                legend=True, 
                have_y_label=True
            )
            for objective_name in obj_names
        ]

In [37]:
preference_request.response = pd.DataFrame(
        [(1661.69, 7.09049, 0.07099)],
        columns=preference_request.content["dimensions_data"].columns,
    )

In [38]:
step_max = 47

while method.steps_taken < step_max:
    ranges_plot_request, _, preference_request = method.iterate(preference_request)
    ranges_plot_request.content["total_steps"] = 80
    nav_figures = [
                extend_navigator_plot(ranges_plot_request, objective_name, nav_figure)
                for nav_figure, objective_name in zip(nav_figures, obj_names)
            ]

In [39]:
preference_request.response = pd.DataFrame(
        [(1666.60, 7.09049,0.07099)],
        columns=preference_request.content["dimensions_data"].columns,
    )

In [40]:
step_max = 80

while method.steps_taken < step_max:
    ranges_plot_request, _, preference_request = method.iterate(preference_request)
    ranges_plot_request.content["total_steps"] = 80
    nav_figures = [
                extend_navigator_plot(ranges_plot_request, objective_name, nav_figure)
                for nav_figure, objective_name in zip(nav_figures, obj_names)
            ]

KeyboardInterrupt: 

In [41]:
nav_figures[2]

In [42]:
nav_figures = [
    nav_figure.update_layout(font_size=15)
    for nav_figure, objective_name in zip(nav_figures, obj_names)
]
[
    nav_figure.write_image(f"images/{objective_name}_2.pdf")
    for nav_figure, objective_name in zip(nav_figures, obj_names)
]

[None, None, None]

In [43]:
x_new = np.atleast_2d(
        pfe(
            problem,
            optimistic_data,
            ideal_current,
            nadir_current,
            reference_point=np.asarray((1664.60, 7.09049,0.07099)),
        ).result.xbest
    )

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=931929, Thu Nov 19 20:11:40 2020)


In [44]:
y_new = vehicle_crash_worthiness(x_new)
y_new.tolist()

[[1667.9289344130302, 7.326228240155263, 0.08775118818093706]]

In [45]:
data

,x1,x2,x3,x4,x5,f1,f2,f3
0,1.630568,1.164042,1.044959,2.597417,2.597592,1683.233620,10.854482,0.081062
1,1.174863,1.351114,2.935511,1.768088,2.341108,1683.685034,8.638321,0.112737
2,2.958312,2.926587,1.768924,2.099489,1.687858,1685.865477,9.512983,0.103124
3,1.046416,2.294535,2.940803,2.807601,2.762791,1695.502498,10.406023,0.116614
4,1.907029,1.670924,1.324802,1.098803,1.390753,1669.391926,8.573261,0.115932
...,...,...,...,...,...,...,...,...
97,1.258890,2.478506,1.689111,1.451819,2.653414,1679.755866,8.441823,0.116881
98,2.600078,2.086669,2.566626,1.802794,1.138428,1681.976194,8.856274,0.170954
99,1.860919,1.898737,2.611184,2.668343,1.866846,1689.929966,10.352398,0.125945
100,1.017108,2.991566,1.013929,1.085325,2.201112,1672.447135,7.233944,0.081460


In [46]:
x = data[var_names].values
y = data[obj_names].values
x = np.vstack((x, x_new))
y = np.vstack((y, y_new))
data = np.hstack((x, y))
data = pd.DataFrame(data, columns=var_names + obj_names)

In [47]:
data

,x1,x2,x3,x4,x5,f1,f2,f3
0,1.630568,1.164042,1.044959,2.597417,2.597592,1683.233620,10.854482,0.081062
1,1.174863,1.351114,2.935511,1.768088,2.341108,1683.685034,8.638321,0.112737
2,2.958312,2.926587,1.768924,2.099489,1.687858,1685.865477,9.512983,0.103124
3,1.046416,2.294535,2.940803,2.807601,2.762791,1695.502498,10.406023,0.116614
4,1.907029,1.670924,1.324802,1.098803,1.390753,1669.391926,8.573261,0.115932
...,...,...,...,...,...,...,...,...
98,2.600078,2.086669,2.566626,1.802794,1.138428,1681.976194,8.856274,0.170954
99,1.860919,1.898737,2.611184,2.668343,1.866846,1689.929966,10.352398,0.125945
100,1.017108,2.991566,1.013929,1.085325,2.201112,1672.447135,7.233944,0.081460
101,1.013009,2.983837,1.013699,1.049942,1.016079,1666.864823,7.064630,0.092512


In [48]:
problem = DataProblem(
    data=data,
    objective_names=obj_names,
    variable_names=var_names,
    bounds=bounds
)
problem.train(GaussianProcessRegressor, {"kernel":Matern(nu=3/2)})

optimizer = RVEA(
    problem,
    use_surrogates=True,
    **{"n_iterations": 10, "n_gen_per_iter": 20, "selection_type": "optimistic"}
)

while optimizer.continue_evolution():
    optimizer.iterate()
    
individuals = optimizer.population.individuals

objectives = (
        optimizer.population.objectives - optimizer.population.uncertainity
    )
    
non_dom_indices = nds(objectives)[0][0]

individuals = individuals[non_dom_indices]
objectives = objectives[non_dom_indices]
optimistic_data = pd.DataFrame(
    np.hstack((individuals, objectives)),
    columns=problem.variable_names + problem.objective_names,
)

method = ONAUTILUS(
    known_data=data[obj_names].values,
    optimistic_data=optimistic_data[obj_names].values,
    objective_names=obj_names,
    max_multiplier=[1,1,1],
    num_steps=100
)

ranges_plot_request, _, preference_request = method.requests()
ranges_plot_request.content["total_steps"] = 80

nav_figures = [
            create_navigator_plot(
                ranges_plot_request,
                objective_name,
                have_x_label=True, 
                legend=True, 
                have_y_label=True
            )
            for objective_name in obj_names
        ]

In [49]:
preference_request.response = pd.DataFrame(
        [(1665.60, 7.09049,0.075)],
        columns=preference_request.content["dimensions_data"].columns,
    )

In [50]:
step_max = 80
prev_step=100
while method.steps_taken < step_max:
    if method.steps_taken == prev_step:
        break
    prev_step=method.steps_taken
    ranges_plot_request, _, preference_request = method.iterate(preference_request)
    ranges_plot_request.content["total_steps"] = 80
    nav_figures = [
                extend_navigator_plot(ranges_plot_request, objective_name, nav_figure)
                for nav_figure, objective_name in zip(nav_figures, obj_names)
            ]

In [51]:
method.steps_taken

80

In [52]:
nav_figures[0]

In [53]:
soln = method.request_solutions_plot()

In [54]:
soln.content['achievable_ids']

array([6])

In [55]:
soln.content['achievable_ids_opt']

array([30])

In [56]:
soln.content['data']

,f1,f2,f3
0,1693.608278,11.287624,0.058335
1,1684.267352,9.806435,0.074970
2,1680.733597,10.116844,0.070586
3,1685.353278,10.223600,0.067941
4,1672.447135,7.233944,0.081460
5,1666.864823,7.064630,0.092512
6,1667.928934,7.326228,0.087751


In [57]:
nav_figures = [
    nav_figure.update_layout(font_size=15)
    for nav_figure, objective_name in zip(nav_figures, obj_names)
]
[
    nav_figure.write_image(f"images/{objective_name}_3.pdf")
    for nav_figure, objective_name in zip(nav_figures, obj_names)
]

[None, None, None]